# Projet Archi BigData

## équipe
- Marwa Iben Khalifa
- Maxime Cordeiro
- Louis Poulin
- Ivan Braga Fernandes
- Hicham Hecharif Chefchaouni

## client
Binance

# contraintes
- serveur mongodb
- cluster Hadoop HDFS
- processing des données avec Apache Spark

# livrables demandés
- un support de présentation (ex: powerpoint)
- méthodologie
- présentation du sujet + source de donnée
- présentation de l’architecture (ex: draw io)
- justification des choix de technos
- difficultés rencontrées, axe d’amélioration

## Chargement de spark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window
#from mongoDbConnection import instert_data


In [5]:
spark = SparkSession.builder.getOrCreate()
path = "hdfs://localhost:9000/data/BNBBUSD*"

## ajout du header

In [6]:
schema = StructType() \
        .add("timestamp",StringType(),True) \
        .add("open",FloatType(),True) \
        .add("high",FloatType(),True) \
        .add("low",FloatType(),True) \
        .add("close",FloatType(),True) \
        .add("volume",FloatType(),True) \
        .add("close_time",StringType(),True) \
        .add("quote_asset_volume",FloatType(),True) \
        .add("number_of_trades",IntegerType(),True) \
        .add("taker_buy_base_asset_volume",FloatType(),True) \
        .add("taker_buy_quote_asset_volume",FloatType(),True) \
        .add("ignore",IntegerType(),True)
      
df_binance = spark.read.format("csv") \
      .option("header", False) \
      .schema(schema) \
      .load(path)
df_binance.show(5)

+-------------+--------+--------+--------+--------+---------+-------------+------------------+----------------+---------------------------+----------------------------+------+
|    timestamp|    open|    high|     low|   close|   volume|   close_time|quote_asset_volume|number_of_trades|taker_buy_base_asset_volume|taker_buy_quote_asset_volume|ignore|
+-------------+--------+--------+--------+--------+---------+-------------+------------------+----------------+---------------------------+----------------------------+------+
|1618185600000| 525.443|551.6302|  514.65|550.8761|355754.25|1618199999999|      1.89119728E8|          122444|                  171540.98|                 9.1285928E7|     0|
|1618200000000|550.7883|   584.0|546.2314|580.2622| 463781.9|1618214399999|      2.62247936E8|          210765|                  239809.27|                1.35662352E8|     0|
|1618214400000|  580.32|   638.0|   533.0|593.3092|1097510.2|1618228799999|       6.4061562E8|          474525|         

In [23]:
import json
df_binance = df_binance.withColumn("_id", col("timestamp"))

json_data = df_binance.toJSON().map(lambda j: json.loads(j)).collect()
#instert_data('BNBBUSD', json_data)

In [24]:
json_data

[{'timestamp': '1618185600000',
  'open': 525.443,
  'high': 551.6302,
  'low': 514.65,
  'close': 550.8761,
  'volume': 355754.25,
  'close_time': '1618199999999',
  'quote_asset_volume': 189119728.0,
  'number_of_trades': 122444,
  'taker_buy_base_asset_volume': 171540.98,
  'taker_buy_quote_asset_volume': 91285928.0,
  'ignore': 0,
  '_id': '1618185600000'},
 {'timestamp': '1618200000000',
  'open': 550.7883,
  'high': 584.0,
  'low': 546.2314,
  'close': 580.2622,
  'volume': 463781.9,
  'close_time': '1618214399999',
  'quote_asset_volume': 262247936.0,
  'number_of_trades': 210765,
  'taker_buy_base_asset_volume': 239809.27,
  'taker_buy_quote_asset_volume': 135662352.0,
  'ignore': 0,
  '_id': '1618200000000'},
 {'timestamp': '1618214400000',
  'open': 580.32,
  'high': 638.0,
  'low': 533.0,
  'close': 593.3092,
  'volume': 1097510.2,
  'close_time': '1618228799999',
  'quote_asset_volume': 640615620.0,
  'number_of_trades': 474525,
  'taker_buy_base_asset_volume': 547766.3,
  

In [60]:
df_binance.write.saveAsTable("binance")

In [57]:
df1=spark.sql("select * from binance limit 5")
df1.show()

+-------------+--------+--------+--------+--------+---------+-------------+------------------+----------------+---------------------------+----------------------------+------+
|    timestamp|    open|    high|     low|   close|   volume|   close_time|quote_asset_volume|number_of_trades|taker_buy_base_asset_volume|taker_buy_quote_asset_volume|ignore|
+-------------+--------+--------+--------+--------+---------+-------------+------------------+----------------+---------------------------+----------------------------+------+
|1669075200000|15761.55|15955.62|15726.98|15828.25|20762.799|1669089599999|      3.28859616E8|          491909|                  10147.984|                  1.607348E8|     0|
|1669089600000|15828.28|15862.66| 15598.6|15742.88|20869.232|1669103999999|      3.28515104E8|          452242|                  10223.968|                 1.6093504E8|     0|
|1669104000000|15742.88|15805.61|15638.63| 15730.0|20420.682|1669118399999|       3.2082464E8|          451824|         